## Проект: RND Exploratory Behavior

**Команда: Сергей Червонцев, Иван Провилков**


Мы исследуем exploration поведение агента используя "Random Network Distillation (RND)" (https://arxiv.org/abs/1810.12894). 

Интересно узнать, как ведет себя агент не имея награды от среды, когда он пользуется только своим внутренним "интересом", который обеспечивается с помощью RND. Меньше ли получается итоговое качество, сколько времени занимает такое обучение и какие факторы на него влияют?

### Мы поставили себе задачу написать агента, воспроизвести результаты статьи про RND на игре "Montezuma revenge" и посмотреть на поведение агента без награды от среды. 

### Мы обнаружили, что данная среда и алгоритм работают очень долго. В дальнейшем (после сдачи проекта) мы собираемся продолжать эксперименты с этой средой. Поэтому одной из своих главных задач на проект мы сделали ускорение обучения алгоритма в этой среде. Чтобы эксперименты занимали меньше времени на имеющихся ресурсах (2 сервера по 2 1080ti).

### Описание среды:

Montezuma revenge это игра Atari, в которой агент должен пройти через множество комнат (всего их 99) в лабиринте, чтобы найти сокровище. Определенные комнаты закрыты дверями и чтобы туда попасть нужно найти и подобрать ключ. В комнатах есть монстры, которые убивают персонажа и их нужно избегать. Также персонаж умирает от падения с высоты или попадания в ловушку (падение в лаву). В комнатах есть различные алмазы, которые игрок может подбирать и получить награду, а также снаряжение, которое можно надеть (факелы, мечи...). У персонажа есть 5 жизней, то есть он может 5 раз умереть до начала новой игры.

Состояния среды: RGB Картинка 210x160 пикселей (мы рескейлим до 84х84)

Пространство действий: агент может бегать в разные стороны, прыгать и использовать предметы, всего 18 Discrete действий. Полное описание действий можно найти  здесь (https://github.com/openai/gym/blob/2ec4881c22b129d1f06173d136529477c0d8d975/gym/envs/atari/atari_env.py#L219)

Extrinsic Reward: даётся за подбор ключа, открытие двери, подбор алмазов в комнатах, убийство монстров оружием.


### Модель

Мы воспроизводим модель из статьи "Random Network Distillation (RND)". Используем Pytorch. 

### Гиперпараметры:

Все гиперпараметры эксперимента можно посмотреть в config.yaml.

In [26]:
!cat config.yaml

EnvName: MontezumaRevengeNoFrameskip-v4
MaxStepsPerEpisode: 4500
ImageHeight: 84
ImageWidth: 84
UseStickyAction: True
StickyActionProb: 0.25

NumWorkers: 128
RolloutSteps: 128
NumInitSteps: 2048

ExtRewardDiscount: 0.999
IntRewardDiscount: 0.99
ExtCoeff: 2.0
IntCoeff: 1.0
LearningRate: 0.0001
ClipGradNorm: 10.0
BatchSize: 4096
EpochSteps: 4
SavePath: "/home/iv-provilkov/lightning_rnd/only_intrinsic_fast.ckpt"

NumEpochs: 10000

RNDUpdateProportion: 0.25
PPOEntropyCoeff: 0.001
PPORewardEps: 0.1
PPOAdvLambda: 0.95
UseVTraceCorrection: True

UseTPU: False


#### Некоторые гиперпараметры:

UseStickyAction: True -- использовать ли повторение предыдущего действия

StickyActionProb: 0.25 -- вероятность того, что повторится предыдущее действие

NumWorkers: 128 -- количество воркеров (процессов) в которых параллельно будет происходить сбор статистик для обучения. 

RolloutSteps: 128 -- количество шагов делаемых каждым воркером для сбора статистики

NumInitSteps: 1024 -- количество стартовых шагов.

ExtCoeff (extrinsic coefficient): -- коэффициент отвечающий за то как сильно мы учитываем награду от среды.

IntCoeff (intrinsic coefficient): -- коэффициент отвечающий за то как сильно мы учитываем внутреннюю награду (Random Network).

UseVTraceCorrection: -- использовать ли V-Trace

UseTPU -- в разработке

### Результаты запусков:

### 1) Ext Coeff: 2.0, Int Coeff 1.0

**ExtRew per Rollout:**
![](./log_screens/usual_2_reward_per_rollout.png)

**IntRew per Rollout:**
![](./log_screens/usual_2_int_reward_per_rollout.png)

**ExtRew per Episode (we log 1 worker with fixed index, total there are 128 workers):**
![](./log_screens/usual_2_reward_per_epi.png)

**Example of work (checkpoint from 5k steps):**
![](./videos/usual_training_2.gif)


### ExtCoeff: 1.0, IntCoeff: 1.0
#### Понижение влияния Extrinsic Reward (награды от среды)

**ExtRew per Rollout:**
![](./log_screens/usual_1_reward_per_rollout.png)

**IntRew per Rollout:**
![](./log_screens/usual_1_int_reward_per_rollout.png)

**ExtRew per Episode:**
![](./log_screens/usual_1_reward_per_epi.png)

**Example of work (checkpoint from 10k steps):**
![](./videos/usual_1.gif)

### ExtCoeff 0.0, IntCoeff 1.0
#### Оставим только Intrinsic Reward (RND)

**ExtRew per Rollout:**
![](./log_screens/only_i_reward_per_rollout.png)

**IntRew per Rollout:**
![](./log_screens/only_i_int_reward_per_rollout.png)

**ExtRew per Episode (model does not train on it):**
![](./log_screens/only_i_reward_per_epi.png)

**Чекпоинт с этого запуска не сохранился, поэтому без видео**

#### Видим, что когда модель не наблюдает награду от среды, то эта награда увеличивается очень медленно (модель никто не поощряет за взятие алмазов). Также интересно то, что Intrinsic Reward после 3000 шагов начинает в среднем медленно расти. То есть модель понемногу учится исследовать что-то новое. 

### Ускорение

#### В базовой модели, которую мы написали, обучение на 10.000 шагов занимает около 3х дней. Причем дальнейшее ускорение вширь (увеличивая количество cpu или gpu) сделать трудно, так как увеличение NWorkers не ускоряет обучение моделей, как и увеличение размера сети или батча. Мы замерили, что в такой реализации на работу сбора статистик в environment-е тратится около 60% времени работы основного процесса (при том что этот сбор делается сразу 128 воркерами). В это время обучение сети на ГПУ не происходит. Мы решили распараллелить два этих процесса, чтобы обучение происходило одновременно со сбором статистик на старых (отстающих на 1 апдейт) весах сети (как в IMPALA).

**Результаты ускорения:**

### ExtCoeff 0.0, IntCoeff 1.0 (parallel env and training)

**ExtRew per Rollout:**
![](./log_screens/only_int_fast_reward_per_rollout.png)

**IntRew per Rollout:**
![](./log_screens/only_int_fast_int_reward_per_rollout.png)

**ExtRew per Episode (model does not train on it):**
![](./log_screens/only_int_fast_reward_per_epi.png)

**Example of work:**
![](./videos/only_intrinsic_fast.gif)

**На этом видео агент научился долго не умирать. При этом интересно, что он заходит во вторую комнату и начинает "играться с летающими черепами, а как-только их там не становится, то больше он туда не хочет заходить. Подобную игру с черепами можно наблюдать и на видео с ExtRew 1, IntRew 1 выше. Скорее всего это происходит потому, что прыгая вокруг черепов он постоянно получает новые картинки, к которым сеть еще не привыкла. За эти новые картинки intrinsic reward награждает его, так как агент еще не успел выучить ответ случайной сети на них.**

**Также, судя по графикам обучения, то, что мы делаем подсчет статистик на старых (отстающих на 1 апдейт) весах влияет на обучение, потому что за то же количество эпох агент достиг меньшей награды и продвинулся не так далеко.**

**Для того чтобы с этим побороться мы решили добавить V-Trace, однако в наших запусках с ним пока получалось только хуже. Почему так происходит, мы пока не поняли.**

### Результаты ускорения.

Результаты обучения на 10к шагов, 128 воркеров.

| Модель |  Время (часов)  | Coeff |
|------|------|------|
|  Базовая модель| 84| x1|
|  Модель с раздельным env_runner и trainer| 33| x2.5|

Таким образом, мы смогли получить ускорение в 2.5 раза.

# Результаты

**Нам удалось воспроизвести статью Random Network Distillation.**

**Наши эксперименты показали, что с помощью RND агент может учиться без награды от среды. Однако, это занимает больше времени, а также ему труднее попасть в новые комнаты.**

**Мы заметили, что из-за RND награды агент может бегать вокруг движущихся объектов (черепов). Картинка меняется и агент получает немного больший ревард. С этой проблемой как-то надо бороться. В случае с движущимися монстрами, агент умирает от своей беготни вокруг монстров. Поэтому можно добавить отрицательный ревард за потерю жизни.**

**Мы реализовали параллельное выполнение env_runner-а, который собирает статистики, и процесса обновления весов. Это дало нам ускорение в 2.5 раза.**

**Мы реализовали V-Trace, однако результаты с ним у нас пока были хуже, чем без него. Проблему пока не нашли.**

### Планы на будущее

**Понять в чем проблема с V-Trace**

**Улучшить алгоритм обучения в данной среде за счет добавления других способов предсказания uncertainty агента для действий и наблюдений.**

## Инструкции по запуску кода

### Для запуска модели нужно установить нужные библиотеки, мы используем python3.6 и cuda9.2

In [14]:
!pip install -r requirements.txt

Установка torch

In [15]:
!pip install torch==1.5.0+cu92 torchvision==0.6.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

### Запуск агента с обученными весами

1. Скачиваем веса из google drive с помощью wget (ниже) или руками: https://drive.google.com/drive/folders/15RBD-BrWUlylYLR13u7NWRhnMTAb9cWs?usp=sharing
2. Пишем путь до весов в config.yaml. 
Пример: SavePath: "/home/chervontsev/checkpoints/usual_2.ckpt", выбираем надо ли оставлять UseStickyAction, мы по умолчанию оставляли. 
3. Запускаем eval.py скрипт: python eval.py. В логах скрипта будет: шаг с которого взят чекпоинт, ревард среды за игру, список посещенных комнат.
4. Видео игры агента появится в папке MontezumaRevengeNoFrameskip-v4_example_run в корне проекта.

In [22]:
# usual_2.ckpt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XyO39st3QnCg5NnQ__OTH-ohwWA93YAx' -O usual_2.ckpt

--2020-05-28 21:33:17--  https://docs.google.com/uc?export=download&id=1XyO39st3QnCg5NnQ__OTH-ohwWA93YAx
Resolving docs.google.com (docs.google.com)... 2a00:1450:4010:c01::c2, 64.233.162.194
Connecting to docs.google.com (docs.google.com)|2a00:1450:4010:c01::c2|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ddul2fqhs1g7fjud52ah7n359rau0rap/1590690750000/01498080638632719806/*/1XyO39st3QnCg5NnQ__OTH-ohwWA93YAx?e=download [following]
--2020-05-28 21:33:22--  https://doc-00-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ddul2fqhs1g7fjud52ah7n359rau0rap/1590690750000/01498080638632719806/*/1XyO39st3QnCg5NnQ__OTH-ohwWA93YAx?e=download
Resolving doc-00-18-docs.googleusercontent.com (doc-00-18-docs.googleusercontent.com)... 2a00:1450:4010:c02::84, 74.125.205.132
Connecting to doc-00-18-docs.googleusercontent.com (doc-00-18-docs.g

In [27]:
# usual_1.ckpt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1WdABk7j7Bs-1_rdp7wU6mfy6gnv5M0UO'\
    -O usual_1.ckpt

--2020-05-29 11:41:49--  https://docs.google.com/uc?export=download&id=1WdABk7j7Bs-1_rdp7wU6mfy6gnv5M0UO
Resolving docs.google.com (docs.google.com)... 2a00:1450:4010:c0b::c2, 173.194.220.194
Connecting to docs.google.com (docs.google.com)|2a00:1450:4010:c0b::c2|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t0au88bc1na79l1a2c5l6vrie92a3l15/1590741675000/01498080638632719806/*/1WdABk7j7Bs-1_rdp7wU6mfy6gnv5M0UO?e=download [following]
--2020-05-29 11:41:51--  https://doc-0s-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t0au88bc1na79l1a2c5l6vrie92a3l15/1590741675000/01498080638632719806/*/1WdABk7j7Bs-1_rdp7wU6mfy6gnv5M0UO?e=download
Resolving doc-0s-18-docs.googleusercontent.com (doc-0s-18-docs.googleusercontent.com)... 2a00:1450:4010:c09::84, 173.194.222.132
Connecting to doc-0s-18-docs.googleusercontent.com (doc-0s-18-docs

In [25]:
# only_intrinsic_fast.ckpt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=16UteZvE_4bcciNiCB4jK7EG_h1crHLeo'\
    -O only_intrinsic_fast.ckpt

--2020-05-28 21:42:39--  https://docs.google.com/uc?export=download&id=16UteZvE_4bcciNiCB4jK7EG_h1crHLeo
Resolving docs.google.com (docs.google.com)... 2a00:1450:4010:c01::c2, 64.233.162.194
Connecting to docs.google.com (docs.google.com)|2a00:1450:4010:c01::c2|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q8k7o0c761eha360s6i831rlp319ermp/1590691350000/01498080638632719806/*/16UteZvE_4bcciNiCB4jK7EG_h1crHLeo?e=download [following]
--2020-05-28 21:42:42--  https://doc-0k-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q8k7o0c761eha360s6i831rlp319ermp/1590691350000/01498080638632719806/*/16UteZvE_4bcciNiCB4jK7EG_h1crHLeo?e=download
Resolving doc-0k-18-docs.googleusercontent.com (doc-0k-18-docs.googleusercontent.com)... 2a00:1450:4010:c02::84, 74.125.205.132
Connecting to doc-0k-18-docs.googleusercontent.com (doc-0k-18-docs.g

In [21]:
!python eval.py

N_updates 5000
Finished, total reward is 4600
All visited rooms: {0, 1, 4, 10, 11, 18, 19, 20}


### Запуск обучения:

1) Настраиваем config.yaml

2) Делаем python train.py